In [1]:
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import math as m
import warnings
warnings.filterwarnings('ignore')

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this Jupyter notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
''')

![title](data/TITLE.png)

### <h1><center>Module 12: Introduction to Z-transforms</center></h1>

To date we have looked at a number of transforms (i.e., Fourier Transform, Discrete Fourier Fourier) that are useful approaches for examining signals in a different domain.  The purpose of this set of notes is to introduce a powerful **generalization** of these: the **Z-transform**.  Studying this transform will allow us to introduce new tools for filtering signals (e.g., low-pass, high-pass, band-pass and band-reject filters).

## Z-transforms
The [Z-Transform](https://en.wikipedia.org/wiki/Z-transform) is used to convert a **discrete-time** signal (i.e., a sequence of real or complex numbers) into a complex frequency-domain representation.  The Z-Transform, here written symbolically as $\mathcal{Z}[\cdot]$, is commonly given as the **bilateral** or **two-sided** transformation of a discrete time-series $x[n]$ into the [formal power series](https://en.wikipedia.org/wiki/Formal_power_series) $X(z)$ defined as:

<div class="alert alert-info" role="alert">
$$
X(z) = \mathcal{Z} \left\{ x[n] \right\} = \sum_{n=-\infty}^{\infty} x[n] z^{-n}, \tag{1}$$ 
</div>

where $n$ is an integer and $z$ is in general a complex number $ z=r\mathrm{e}^{i\phi} = r \left( \mathrm{cos}\,\phi + i\,\mathrm{sin}\,\phi \right).$  For clarity let's explicitly put this expression into the function:

$$
X(z) = \mathcal{Z} \left\{ x[n] \right\} = \sum_{n=-\infty}^{\infty} x[n] r^{-n}e^{-i\phi n}, \tag{2}$$


In some cases - and particularly for causal signals (i.e., $x[n]=0$ for $n\lt 0$), one often finds the **unilateral or one-sided Z-Transform**:

<div class="alert alert-info" role="alert">
$$
X(z) = \mathcal{Z} \left\{ x[n] \right\} = \sum_{n=0}^{\infty} x[n] z^{-n} = \sum_{n=0}^{\infty} x[n] r^{-n}e^{-i\phi n}, \tag{3}$$
</div>

which is commonly used for evaluating the unit impulse response of a discrete-time **causal** system.  

### Connection with Discrete Fourier Transform

An interesting immediate question is "What is the connection between the Z-transform and the DFT we studied previously?".  The answer is the that Z-transform **reduces** to the DFT for scenarios obeying the following three conditions:

1. A signal where $x[n]=0$ for $n<0$ and $n>N$;
2. A $\phi$ defined as  $\phi=2\pi k/N$; and
3. A $r$ value defined as $r=1$.

Given these three conditions, we **exactly** recover the **Discrete Fourier Transform (DFT)**.  Thus, the **Z-Transform** can be seen as a generalization of the **DFT**.  It turns out that this generalization is very powerful and leads to a whole range of tools that are useful for filtering!

**Geophysical Definition**: In geophysics the Z-transform is commonly described as a power series in $z$ as opposed to $z^{-1}$.   While the two equations are equivalent, the do result in a number of changes that I will mention these below. Because most of the DSP literature uses the  former definition, I will use this terminology in this section.

## Example 1 - Unit Delay

**Q:** What happens when we take the Z-transform of signal $x[n]$ ($n\ge0$) that has been delayed by $k$ samples (i.e., $x[n-k]$)?

**A:** Let's evaluate this using the definition of the unilateral or one-sided Z-transform.

$$
\begin{array}{ll}
\mathcal{Z}\left\{x[n-k]\right\}
&=&
\sum_{n=0}^{\infty} x[n-k]z^{-n}  \\
\,&=&
\sum_{j=-k}^{\infty} x[j]z^{-n}\quad \quad j=n-k  \\
\,&=&
\sum_{j=-k}^{\infty} x[j]z^{-(j+k)} \\
\,&=&
\sum_{j=-k}^{\infty} x[j]z^{-j}z^{-k}  \\
\,&=&
z^{-k}\sum_{j=-k}^{\infty} x[j]z^{-j} \\
\,&=&
z^{-k}\sum_{j=0}^{\infty} x[j]z^{-j}\quad\quad x[j]=0, j<0  \\
\,&=&
z^{-k}X(z)  \\
\end{array}\tag{4}
$$

where $X(z)$ is the Z-transform of $x[n]$.  Thus, $z^{-k}$ can be interpreted as an operator that **delays** a sequence by $k$ samples. Correspondingly, $z^k$ is an operator that **advances** a sequency by $k$ samples.

A consequence of this interpretation is that it can be used to represent regularly sampled time-series. For example, if we have a sequence of numbers $x[n]=[2,4,6,8,10]$ for $n=[0,1,2,3,4]$ then we can think about representing this sequence as the following $X(z)$ time-series:

$$X(z) = 2z^{0} + 4z^{-1} + 6z^{-2} + 8z^{-3} + 10z^{-4}, \tag{5a} $$ 

where the nth term in the advancing time series corresponds to the power $z^{-n}$. 

## Linear Constant Cofficient Difference Equations (LCCDE) 

Previously in the class we mentioned linear constant coefficient difference equations (LCCDE) in the context of linear time-invariant (LTI) systems.  In any LTI system, its input $x[n]$ and output $y[n]$ can be related via a Nth order linear constant coefficient [difference equation](https://en.wikipedia.org/wiki/Linear_difference_equation):

$$ \sum_{k=0}^{N} a_k y[n-k] = \sum_{k=0}^{M} b_k x[n-k]. \tag{5b} $$

Let's explore what happens when we apply the Z-transform concept that we discussed above.  Apply this to both side of equation 5 yields the following

$$\mathcal{Z} \left[ \sum_{k=0}^{N} a_k y[n-k] \right] 
=  
\mathcal{Z} \left[ \sum_{k=0}^{M} b_k x[n-k] \right]. \tag{6a}$$

However, because this is a linear operation we may write

$$ \sum_{k=0}^{N} a_k \mathcal{Z} \left[ y[n-k] \right] 
=  
\sum_{k=0}^{M} b_k \mathcal{Z} \left[  x[n-k] \right]. \tag{6b}$$

and then apply the definition of the Z-transform as the unit delay operator

$$\sum_{k=0}^N a_k z^{-k}Y(z) =  \sum_{k=0}^{M} b_k z^{-k} X(z). \tag{6c}$$

Realizing that $Y(z)$ and $X(z)$ do not depend on $k$ allows us to write:

$$Y(z) \sum_{k=0}^N a_k z^{-k} = X(z) \sum_{k=0}^{M} b_k z^{-k}. \tag{7}$$

Let's now rewrite this equation by dividing both sides of equation 7 by $X(z)$ and by the series on the left.  This results in the following expression:

$$H(z) \equiv \frac{Y(z)}{X(z)} =  \frac{\sum_{k=0}^{M} b_k z^{-k}}{\sum_{k=0}^N a_k z^{-k} }, \tag{8a} $$

where $H(z)$ is known as the [transfer function](https://en.wikipedia.org/wiki/Transfer_function) and effectively describes how the input and output are related in Z-space:

$$Y(z) = H(z) X(z). \tag{8b}$$

Applying $H(z)$ may also be thought of as applying a **filtering** operation that transforms an input signal represented by $X(z)$ into an output signal represented by $Y(z)$.  

We have seen something like this before when we discussed the convolution theorem.  In the case where the Z-transform becomes a Fourier Transform the operation in equation 8b effectively represents the **convolution theorem in the frequency domain**.  Thus, this may be thought of as an extension of the convolution theorem to the Z-transform.


### FIR and IIR systems

There are two classes of transfer function that are commonly applied in digital signal processing:

1. A system with coefficients $a_k=0, k\ge 1$ does not have feedback, and is called a **nonrecursive filter** or a [finite-impulse response (FIR)](https://en.wikipedia.org/wiki/Finite_impulse_response) filter.

2. A system with coefficients $a_k\neq 0, k\ge 1$ is said to have **feedback** since the current output value depends on the *previous* output values.  A filter exhibiting such a characteristic is called both a **recursive filter** or an [infinite-impulse response (IIR)](https://en.wikipedia.org/wiki/Infinite_impulse_response) filter. 

Let's now make a connection with the LCCDE studied previously: 

1. An averaging filter depends only on the input sequence and not on the previous filtered output and thus an averaging operator may be considered as a FIR filtering operator.

2. Your current bank account or mortage balance depends on what is was in months previous and thus compound interest or mortgage ammortization operators may be considered as a recursive IIR filtering operator.

## Fundamental Theorem of Algebra

The [Fundamental Theorem of Algebra](https://en.wikipedia.org/wiki/Fundamental_theorem_of_algebra) states that 

"Every non-zero, single-variable, degree n polynomial with complex coefficients has, counted with multiplicity, exactly n complex roots. The equivalence of the two statements can be proven through the use of successive polynomial division."

What is the implication here? This means that a $n$-order polynomial expanded about some complex value $z_0$,

$$p(z)=\sum_{k=0}^{n}c_k(z-z_0)^k = c_0+c_{1}(z-z_{0})^{1}+c_{2}(z-z_{0})^{2}+\cdots +c_{n}(z-z_{0})^{n}, \tag{9}$$

can be rewritten as

$$p(z)= a_0 \Pi_{k=1}^n (z-z_k) = a_0(z-z_{1})(z-z_{2})\cdots (z-z_{n}). \tag{10}$$

where the $\Pi$ represents Pi summation notation [e.g., $\Pi_{k=3}^{5} k =3 \times 4 \times 5=60$].  Thus, we can write the above transfer function $H(z)$ as:

<div class="alert alert-info" role="alert">
$$H(z) \equiv \frac{Y(z)}{X(z)} =  \frac{ p_0\Pi_{k=1}^{M} (z-p_k)}{ q_0 \Pi_{k=1}^{N}(z-q_k) }. \tag{11} $$
</div>

You may (should!) be concerned about what's going on in the denominator.  In particular, what is happening whenever $z=q_k$: division by zero!  Thus, we must examine the **stability** of these transfer functions as well as determine in which region of the complex plane these series converge.

## Poles and Zeros

Starting from the $H(z)$ defined in equation 11, let's examine a few different important values.

**Zeros:** are the values of $z$ for which $H(z)=0$.  These occur when by $z=p_k$ in equation 11.  These **do not cause** any stability issues; however, they cause certain parts of the output spectrum to be equal to zero.

**Poles:** are the values of $z$ for which $H(z)=\infty$.  These occur when $z=q_k$ in equation 11. These **do cause** stability issues since division by zero will cause an infinite output!

An important concept is that by specifying the locations of where we put $p_k$ and $q_k$ we can design filters (e.g., low-pass, high-pass, band-pass) that will take our input data $x[n]$ and give us the desired output signals $y[n]$ (e.g., low-passed, high-passed or band-passed data).

**Q:** How many zeros and poles are there in equation 11?

**A:**: There are $M$ zeros and $N$ poles.

## Thinking about convergence

Let's first refresh ourselves about the concept of convergence and (infinite) geometric series.  We can start with the following $N$ term **finite geometric series**:

$$y = \sum_{n=0}^{N-1} ax^n = a \left(\frac{1-x^N}{1-x}\right), \tag{11a}$$

where we have used a [geometric sum](https://en.wikipedia.org/wiki/Geometric_progression#Geometric_series) to evaluate on the right hand side. There are no issues here in terms of stability (i.e., $y<\infty$) even at $x=1$ because this just results in a sum of $y=aN$.

What happens when we now consider an **infinite geometric series**?

$$y = \sum_{n=0}^{\infty} ax^n = a \left(\frac{1-x^\infty}{1-x}\right), \tag{11b}$$

Clearly, there is now a different concern because for $y<\infty$ we now have to put restrictions on $|x|\le 1$ because otherwise $x^\infty$ will lead to $y=\infty$. The restriction $-1\le x \le 1$ is thus the **region of convergence** of this infinite sum along the real $x$ axis.

What happens if we now have the follow **infinite geometric series**?

$$y = \sum_{n=0}^{\infty} (ax)^n = a \left(\frac{1-(ax)^\infty}{1-(ax)}\right). \tag{11c}$$

In this case we see that $|ax|<1$ or better yet $|a||x|<1$.  Thus, for this to be stable (i.e., $y<\infty$) we need to have $|x|< |a|^{-1}$. 

## Region of Convergence

The region of convergence (ROC) indicates when Z-transforms of a sequence converges.  Generally, there exists some $z$ such that 

$$\left| X(z) \right| =\left|\sum_{k=-\infty}^{\infty} x[n]z^{-n}\right|\rightarrow \infty \tag{12}$$

where the Z-transform **does not converge**.  The set of values for $z$ for which $X(z)$ converges,

<div class="alert alert-info" role="alert">
$$\left| X(z) \right| =\left|\sum_{k=-\infty}^{\infty} x[n]z^{-n}\right| \le \sum_{k=-\infty}^{\infty} \left|x[n]z^{-n}\right|   < \infty \tag{13}$$
</div>

is called the ROC.  The ROC must be specified along with $X(z)$ in order for the Z-transform to be considered "complete".

Assuming that $x[n]$ is of infinite length, let's decompose $X(z)$ as the following:

$$\begin{eqnarray}
X(z) &=& X_{-}(z) + X_+(z), \tag{14}
\end{eqnarray}$$

where these two contributions are the anticausal ($X_-(z)$) and causal ($X_+(z)$) components of $X(z)$:

$$\begin{eqnarray}
X_{-}(z) &=& \sum_{n=-\infty}^{-1}x[n]z^{-n} \tag{15a}\\
X_{+}(z) &=& \sum_{n=0}^{\infty}x[n]z^{-n} \tag{15b}\\
\end{eqnarray}$$

where it is clear that the sum of equations 15a and 15b satisfy equation 14.

### Convergence of $X_+(z)$

For a series to converge, the series

$$\begin{eqnarray}
X_+(z) &=& x[0]z^{-0} + x[1]z^{-1} +\dots + x[n]z^{-n}+ ... \tag{16a}\\
\,     &=& f_0(z)+f_1(z)+ \dots + f_n(z) + \dots \tag{16b}\\
\end{eqnarray}$$

has to satisfy the following [ratio test](https://en.wikipedia.org/wiki/Ratio_test) behaviour:

$$\lim_{n\rightarrow\infty} 
\left|
\frac{f_{n+1}(z)}{f_n(z)}
\right|
< 1.\tag{17}
$$

Thus, assuming that the input sequence $x[n]$ converges to a value finite value $R_+$

$$
\lim_{n\rightarrow\infty} 
\left|
\frac{x[n+1]}{x[n]}
\right|
= R_+ \tag{18}
$$

then $X_+[z]$ will converge if

$$
\lim_{n\rightarrow\infty} 
\left|
\frac{x[n+1]z^{-n-1}}{x[n]z^{-n}}
\right|
=
\lim_{n\rightarrow\infty} 
\left|
\frac{x[n+1]}{x[n]}
\right|
\left|z^{-1}\right|
<1 \tag{19}
$$
This implies that

$$\left|z\right| > \lim_{n\rightarrow\infty} 
\left|
\frac{x[n+1]}{x[n]}
\right|=R_+. \tag{20}
$$

That is, the ROC for $X_+(z)$ is 

$$|z|>R_+. \tag{21}$$

### Convergence of $X_-(z)$

Assuming that the sequence converges to a non-infinite value $1/R_-$

$$
\lim_{m\rightarrow\infty} 
\left|
\frac{x[-m-1]}{x[-m]}
\right|
= \frac{1}{R_-}, \tag{22}
$$

the anticausal components will converge if

$$
\lim_{m\rightarrow\infty} 
\left|
\frac{x[-m-1]z^{m+1}}{x[-m]z^{m}}
\right|
=
\lim_{m\rightarrow\infty} 
\left|
\frac{x[-m-1]}{x[-m]}
\right|
\left|z\right| = \frac{1}{R_-} \left|z\right|
<1. \tag{23}
$$

Thus, the ROC for $X_-(z)$ is 

$$|z|<R_-.\tag{25}$$

### Combining the Results

The ROC for a infinite sequence, $X(z)$ is given by 

$$R_+<|z|<R_-. \tag{26}$$

Note that if $R_- < R_+$ then there is **no ROC** and $X(z)$ **does not exist**.

Let's look at this graphically:

<img src="Fig/ROC.png" width="800">

**Figure 1. Illustrating the different regions of convergence for $X_+(z)$, $X_-(z)$ and $X(z)$.**

## Example 2 - Causal geometric sequence

**Q:** Determine the z-transform of $x[n] = a^n u[n]$ where $u[n]$ is the unit step function. 

**A:** The input function may be written as

$$X(z) = \sum_{n=-\infty}^{\infty} a^n u[n]z^{-n} = \sum_{n=0}^{\infty} a^n z^{-n}= \sum_{n=0}^{\infty} \left(a z^{-1}\right)^n. \tag{27}$$

According to the above, $X(z)$ will converge if

$$\sum_{n=0}^{\infty} \left|a z^{-1}\right|^n< \infty \tag{28}$$

Applying the **ratio test**

$$\lim_{n\rightarrow\infty} 
\left|
\frac{f_{n+1}(z)}{f_n(z)}
\right|
< 1. \tag{29}
$$

we get

$$\lim_{n\rightarrow\infty} 
\left|
\frac{a^{n+1}z^{-n-1}}{a^nz^{-n}}
\right|
= 
\left|
az^{-1}
\right|
< 1.\tag{30}
$$

Thus, the convergence condition is that $|a|<|z|$.  Thus, within this region we may use a geometric series to evaluate this infinite summation in the region where the series converges:

$$
X(z) = \sum_{n=0}^\infty \left(az^{-1}\right)^n
=
\frac{1-\left(az^{-1}\right)^\infty}{1-az^{-1}}
=
\frac{1}{1-az^{-1}}
=
\frac{z}{z-a},
\tag{31}
$$
where the second equality is reached in the ROC because $az^{-1}<1$ which means that $(az^{-1})^\infty=0$; the final step is just multiplying top and bottom by $z$.

Thus, together with the ROC, the z-transform of $x[n]=a^n u[n]$ is:

$$X(z) = \frac{z}{z-a}, \quad |a| < |z|. \tag{32}$$

It is clear that $X(z)$ has a **zero** at $z=0$ and a **pole** at $z=a$.

<img src="Fig/ROC_EX2.png" width="500">

**Figure 2. Plotting the ROC of $|z|>|a|$. Note that if $|a|<1$ then the Discrete Fourier Transform is guaranteed to exist. However, if $|a|>1$ then it is not guaranteed to exist (but it may if the input terms  in $u(z)$ fall over faster than $a^{-1}$).**

## Example 3 - Anticausal geometric sequence

**Q:** Determine the z-transform of $x[n]=-b^n u[-n-1]$.

**A:** Let's write

$$X(z) =
-\sum_{n=-\infty}^{\infty}b^n u[-n-1] z^{-n}
= 
-\sum_{n=-\infty}^{-1}b^n z^{-n}\tag{33}
$$

Let's now identify $m=-n$ and write

$$X(z) =
-\sum_{m=1}^{\infty}b^{-m} z^{m}
=
-\sum_{m=1}^{\infty}\left(b^{-1} z\right)^{m}. \tag{34}
$$

Thus, like the Example 2, $X(z)$ converges if $\left|b^{-1}z\right|<1$, or $|b|>|z|$.  This gives:

$$X(z)=-\sum_{m=1}^{\infty} \left(b^{-1}z\right)^m = 
- \frac{b^{-1}z(1-(b^{-1}z)^\infty)}{1-b^{-1}z} =- \frac{b^{-1}z}{1-b^{-1}z} = - \frac{z}{b-z} = \frac{z}{z-b}. \tag{35}$$

Thus, together with the ROC, the z-transform of $x[n]=-b^n u[-n-1]$ is:

$$X(z) = \frac{z}{z-b}, \quad |b|>|z|. \tag{36}$$

Again, it is clear that $X(z)$ has a **zero** at $z=0$ and a **pole** at $z=b$.

<img src="Fig/ROC_EX3.png" width="500">

**Figure 3. Plotting the ROC of $|b|>|z|$. Note that if $|b|>1$ then the Discrete Fourier Transform is guaranteed to exist. However, if $|b|<1$ then it is not guaranteed to exist (but it may if the input terms in $u(z)$ fall over faster than $b^{-1}$).**

## Example 4 - Combining Examples 2 and 3

**Q:** Determine the z-transform of $x[n] = a^n u[n]+b^n u[-n-1]$ where $|a|<|b|$.

**A:**  Employing the previous results we have

$$
\begin{eqnarray}
X(z) &=& \frac{z}{z-a}-\frac{z}{z-b} \quad |a|<|z|<|b| \tag{37a} \\
&=&
\frac{(b-a)z}{(z-a)(z-b)}, \quad |a|<|z|<|b| \tag{37b}\\
\end{eqnarray}
$$

Note that there is still one **zero** ($z=0$) but there are now two **poles** ($z=a$ and $z=b$).  Note that the ROC is given by Figure 1c where $a=R_+$ and $b=R_-$. 

## Example 5 - Unit Impulse 

**Q:** What is the Z-transform of $x[n]=\delta[n-p]$.

**A:** We have

$$X(z) = \sum_{n=-\infty}^\infty \delta[n-p]z^{-n} = z^{-p},\quad |z|<\infty, \tag{38}$$

which effectively states that as long as $r^{-p}<\infty$ the Z transform will converge.

## Example 6 - Finite Geometry Series

**Q:** Determine the z-transform of $x[n]$ which has the form

$$x[n] = \left\{
\begin{array}{cc}
a^n & 0\le n\le N-1 \\
0 & \mathrm {otherwise}\\
\end{array}
\right. \tag{39}
$$

**A:** Assuming that $a^N<\infty$, we have by the geometric series

$$X(z) = \sum_{n=0}^{N-1}\left(az^{-1}\right)^n =
\frac{1-\left(az^{-1}\right)^N}{1-az^{-1}}
=
\frac{1}{z^{N-1}}\frac{z^N-a^N}{z-a}, \quad |z|>0. \tag{40}\\
$$

Note that when $a=z$ then the first summation term reduces to a sum of N ones.

## Finite- and Infinite-duration sequences

**Finite-duration sequence**: A sequence where values of $x[n]$ are non-zero only for a finite time interval.

<img src="Fig/finite.png" width="400">

**Figure 4. Illustrating a number of *finite-duration* sequences and their region of convergence.**

Otherwise $x[n]$ is an **infinite-duration sequence**. There are a number of different types of these infinite-duration sequences:

   * **Right-sided**: If $x[n]=0$ for $n<N_+<\infty$ where $N_+$ is an integer.
    
   * **Left-sided**: If $x[n]=0$ for $n >N_- >-\infty$ where $N_-$ is an integer.
    
   * **Two-sided**: Neither a right- nor left-sided sequence. 

<img src="Fig/infinite.png" width="400">

**Figure 5. Illustrating a number of *infinite-duration* sequences and their region of convergence.**
